In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class Diputados(scrapy.Spider):
    name = "spider_diputados"
    
    def start_requests(self):
        yield scrapy.Request( url = "http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=002", headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, 
                              callback = self.parse )
    
    def parse(self,response):
        dc_dict['nombreDiputado']=response.css('td.textocurrienc > center > strong::text').get()
        dc_dict['tipoElección']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract_first()
        dc_dict['entidad']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[1]
        dc_dict['distrito']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[2]
        # dc_dict['?????']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[3]
        dc_dict['correo']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[4]
        dc_dict['extensión']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[5]
        dc_dict['onomástico']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[6]
        dc_dict['suplente']=response.css('td.textocurri:nth-of-type(2)::text').extract()[6]
        link_asis=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[0]
        link_votaciones=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[1]
        yield response.follow(url=link_asis, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},
                              callback=self.parse2)
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},
                              callback=self.parse4)
    
    def parse2(self,response):
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},
                              callback=self.parse3)
        yield response.follow(url=links[0], headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},
                              callback=self.parse3)
    
    def parse3(self,response):
        prueba = response.xpath('//tr/td/div/font/text()').extract()
        dias = list(filter(lambda a: (a == 'A' or a =='AC'), prueba))
        dc_dict['Asistencias']=dc_dict.get('Asistencias',0) + len(dias)
        

    def parse4(self,response):
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},
                              callback=self.parse5)
        #yield response.follow(url=links[0], headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},
        #                      callback=self.parse5)
                
    def parse5(self,response):
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        aux2={}
        for i in range(1,len(aux),4):
            votos = {}
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i]
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]
            votos['votación']=votacion
            votos['posición']=posi
            aux2['votos']=aux2.get('votos', []) + [votos]
            #dc_dict['votos']=dc_dict.get('votos', []) + [votos]
        dc_dict['votos']=dc_dict.get('votos',0)+len(aux2.get('votos', []))
        


        

In [3]:
dc_dict={}
urlDipu = "http://sitl.diputados.gob.mx/LXIV_leg/"


# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-03-12 19:40:35 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-12 19:40:35 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-03-12 19:40:35 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-12 19:40:35 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-12 19:40:35 [scrapy.extensions.telnet] INFO: Telnet Password: 0cea7d5e021cef88
2021-03-12 19:40:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-12 19:40:35 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

2021-03-12 19:40:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=2&pert=4> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=002)
2021-03-12 19:40:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=2&pert=16> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=002)
2021-03-12 19:40:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=2&pert=1> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=002)
2021-03-12 19:40:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=2&pert=9> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=002)
202

In [4]:
print(dc_dict)

{'nombreDiputado': 'Dip. Elba Lorena Torres Díaz (LICENCIA)', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Aguascalientes ', 'distrito': '2 ', 'correo': 'elba.torres@diputados.gob.mx ', 'extensión': '62100 ', 'onomástico': '15-marzo-1977 ', 'suplente': 'Haydeé Hernández Muñoz\n\t\t\t\t\t\n\n\n ', 'Asistencias': 130, 'votos': 585}
